In [3]:
from matplotlib.patches import ConnectionPatch
import matplotlib.pyplot as plt
import numpy as np
#import scipy.spatial.distance as dist
import mediapipe as mp
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [5]:
def dtp(dist_mat):
    N, M = dist_mat.shape
    cost_mat = np.zeros((N + 1, M + 1))
    for i in range(1,N+1):
        cost_mat[i, 0]  = np.inf
    for j in range(1, M+1):
        cost_mat[0, j] = np.inf
    traceback_mat = np.zeros((N,M))
    for i in range(1,N+1):
        for j in range(1,M+1):
            min_list = [cost_mat[i-1, j-1], # match = 0
                        cost_mat[i-1, j],   #insert = 1
                        cost_mat[i, j-1]]   # deletion = 2
            index_min = np.argmin(min_list)
            cost_mat[i,j] = dist_mat[i -1, j -1] + cost_mat[index_min]
            traceback_mat[i - 1,j - 1] = index_min 
    i = N-1
    j = M -1
    path = [(i,j)]
    while i > 0 or j > 0:
        tb_type = traceback[i,j]
        if tb_type == 0: # đi chéo
            i = i-1
            j = j-1
        elif tb_type == 1: # đi xuống
            i = i - 1
        elif tb_type == 2: # đi ngang
            j = j - 1
        path.append((i,j))
    cost_mat = cost_mat[1:, 1:]
    return path[::-1], cost_mat


In [6]:
def distance_matrix(mat1, mat2):
    N = mat1.shape[1]
    M = mat2.shape[1]
    dist_mat = np.zeros((N,M))
    for i in range(N):
        for j in range(M):
            dist_mat[i, j] = np.linalg.norm(mat1[:, i] - mat2[:,j] )
    return dist_mat
    

In [7]:
def calculate_angle(a,b,c):
    a, b, c = np.array(a), np.array(b),  np.array(c)

    radians = np.arctan2(c[1]- b[1], c[0]-b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180/np.pi)
    if angle > 180:
        angle = 360 - 180
    return angle

In [8]:
def extract_vid(vid_path):
    cap = cv2.VideoCapture(vid_path)
    N = cap.get(cv2.CAP_PROP_FRAME_COUNT)
    matrix = []
    count = 0
    with mp_pose.Pose(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as pose:
        while(cap.isOpened()):
            try:
                ret, frame = cap.read()
                count += 1
            #Change from BGR to RGB
                img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
                img.flags.writeable = False

                #Make detection
                result = pose.process(img)

                # Back the original color channels
                img.flags.writeable = True
                img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

                #Extract landmarks 
                try:
                    landmarks = result.pose_landmarks.landmark
                    #print(landmarks)
                except:
                    pass
                # Landmarks: là 1 list chứa tọa độ 33 điểm trên cơ thể
                theta = []
                for i in range(3):
                    for j in range(i+1, 3):
                        for k in range(j+1,3): 
                            a = [landmarks[i].x, landmarks[i].y]
                            b = [landmarks[j].x, landmarks[j].y]
                            c = [landmarks[k].x, landmarks[k].y]
                            angle = calculate_angle(a, b, c)
                            theta.append(angle)
                matrix.append(theta)

            except:
                pass
            if count == 50:
                break
        
        return np.array(matrix)



In [9]:
mat1 = extract_vid("Sources/dtwvid1.mp4")
mat1

array([[123.64335107],
       [132.96165988],
       [140.23006318],
       [143.29728226],
       [146.99375164],
       [148.23253034],
       [145.18750446],
       [145.19647549],
       [145.46484736],
       [145.8608023 ],
       [144.94529162],
       [149.2129451 ],
       [149.08081394],
       [147.8144634 ],
       [145.68996666],
       [145.84808602],
       [147.30161018],
       [149.75398165],
       [151.69473336],
       [153.71906313],
       [147.8013528 ],
       [147.79114321],
       [147.53394912],
       [146.22250648],
       [147.76496185],
       [146.7680185 ],
       [146.72337288],
       [147.26112316],
       [147.68861427],
       [147.24736224],
       [145.79204956],
       [145.73483983],
       [145.20933763],
       [145.3855727 ],
       [144.69296743],
       [141.05478233],
       [136.64020843],
       [113.60027347],
       [106.25397317],
       [104.39915603],
       [101.79139021],
       [106.29864134],
       [112.59563085],
       [117

In [ ]:
mat2 = extract_vid("Sources/dtw vid2.mp4")
mat2

error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


: 